# Make KE movies (found in supporting information)

In [1]:
import cartopy.crs as ccrs
import cmocean
cmap = cmocean.cm.thermal
cmap.set_bad('w')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from dask.diagnostics import ProgressBar
import warnings
warnings.filterwarnings("ignore")

In [3]:
from yamlparser import YAMLParser, read_parameterized_runs, read_unparameterized_runs

In [4]:
yp = YAMLParser()

In [5]:
## read parameterized runs
exps_param = yp.read('/glade/u/home/noraloose/GL90paper/config_param.yaml')
%time exps_param = read_parameterized_runs(exps_param, read_from_time_list=True)

nw2_0.5deg_N15_KHTH800
nw2_0.5deg_N15_KHGL800_hbbl5
nw2_0.5deg_N15_KHTH800_vary
nw2_0.5deg_N15_KHGL800_vary_hbbl5
nw2_0.5deg_N15_EBT_KHTH800
nw2_0.5deg_N15_EBT_KHGL800_hbbl5
nw2_0.25deg_N15_KHTH300
nw2_0.25deg_N15_KHGL300_hbbl1
nw2_0.25deg_N15_KHGL300_hbbl5
nw2_0.25deg_N15_KHGL300_hbbl20
nw2_0.25deg_N15_KHGL300
nw2_0.25deg_N15_KHTH800
nw2_0.25deg_N15_KHGL800_hbbl5
nw2_0.25deg_N15_KHTH300_vary
nw2_0.25deg_N15_KHGL300_vary_hbbl5
nw2_0.25deg_N15_EBT_KHTH300
nw2_0.25deg_N15_EBT_KHGL300_hbbl5
nw2_0.125deg_N15_KHTH100
nw2_0.125deg_N15_KHGL100_hbbl5
nw2_0.125deg_N15_KHTH800
nw2_0.125deg_N15_KHGL800_hbbl5
nw2_0.125deg_N15_KHTH100_vary
nw2_0.125deg_N15_KHGL100_vary_hbbl5
nw2_0.125deg_N15_EBT_KHTH100
nw2_0.125deg_N15_EBT_KHGL100_hbbl5
CPU times: user 9.63 s, sys: 1.57 s, total: 11.2 s
Wall time: 1min 48s


In [6]:
# read unparameterized runs
exps_unparam = yp.read('/glade/u/home/noraloose/GL90paper/config_unparam.yaml')
%time exps_unparam = read_unparameterized_runs(exps_unparam)

CPU times: user 1.09 s, sys: 412 ms, total: 1.5 s
Wall time: 15.7 s


In [7]:
exp_list = [exps_param, exps_unparam]

In [8]:
for exps in exp_list:

    for exp, v in exps.items():
        ds = v['ds']
        ds['hKE'] = ds['h'] * ds['KE']


In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'font.size': 12}
pylab.rcParams.update(params)

# Movies

In [10]:
pw = 700
ph = 1080
import xmovie
from xmovie import Movie

In [22]:
def plotfunc_KE(ds, fig, tt, framedim='time'):
    vmin = 1e-2
    vmax = 1e3
    unit = r'm$^3$ s$^{-2}$'
    kwargs = {
        'norm': mpl.colors.LogNorm(vmin=vmin, vmax=vmax),
        'cmap': cmap,
        'add_colorbar': False,
    }
    
    ax = fig.subplots(nrows=1, ncols=1, subplot_kw={'projection':ccrs.PlateCarree()})


    KE_snapshot = ds['hKE'].sum(dim='zl').isel({framedim:tt}).compute()

    p = KE_snapshot.plot(ax=ax, **kwargs)
    ax.set_title('Time = %d days' % (5*tt))


    ax.set_extent([0, 60, -70, 70], crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
        

    fig.subplots_adjust(right=0.7, wspace=0.3, hspace=0.3)
    cax = fig.add_axes([0.7, 0.1, 0.03, 0.8])
    cbar = plt.colorbar(p, cax=cax, label=unit, extend='both')
    cbar.ax.tick_params(labelsize=16) 

    fig.suptitle(label)

### 1/2 degree

In [24]:
exp_str = 'nw2_0.5deg_N15_KHTH800'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.5deg_N15_KHTH800.mp4


In [25]:
exp_str = 'nw2_0.5deg_N15_KHGL800_hbbl5'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.5deg_N15_KHGL800_hbbl5.mp4


### 1/4 degree

In [27]:
exp_str = 'nw2_0.25deg_N15_baseline_hmix20'
ds = exps_unparam[exp_str]['ds']
label = exps_unparam[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.25deg_N15_baseline_hmix20.mp4


In [28]:
exp_str = 'nw2_0.25deg_N15_KHTH300'
ds = exps_param[exp_str]['ds'] 
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.25deg_N15_KHTH300.mp4


In [30]:
exp_str = 'nw2_0.25deg_N15_KHTH800'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.25deg_N15_KHTH800.mp4


In [31]:
exp_str = 'nw2_0.25deg_N15_KHGL300_hbbl5'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.25deg_N15_KHGL300_hbbl5.mp4


In [32]:
exp_str = 'nw2_0.25deg_N15_KHGL800_hbbl5'
ds = exps_param[exp_str]['ds'] 
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.25deg_N15_KHGL800_hbbl5.mp4


### 1/8 degree

In [33]:
exp_str = 'nw2_0.125deg_N15_baseline_hmix20'
ds = exps_unparam[exp_str]['ds']
label = exps_unparam[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.125deg_N15_baseline_hmix20.mp4


In [34]:
exp_str = 'nw2_0.125deg_N15_KHTH100'
ds = exps_param[exp_str]['ds'] 
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.125deg_N15_KHTH100.mp4


In [35]:
exp_str = 'nw2_0.125deg_N15_KHTH800'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.125deg_N15_KHTH800.mp4


In [36]:
exp_str = 'nw2_0.125deg_N15_KHGL100_hbbl5'
ds = exps_param[exp_str]['ds'] 
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.125deg_N15_KHGL100_hbbl5.mp4


In [37]:
exp_str = 'nw2_0.125deg_N15_KHGL800_hbbl5'
ds = exps_param[exp_str]['ds']
label = exps_param[exp_str]['label']
mov_custom = Movie(ds, plotfunc=plotfunc_KE, pixelwidth=pw, pixelheight=ph, input_check=False)
mov_custom.save('/glade/u/home/noraloose/GL90paper/movies/KE_%s.mp4' %(exp_str), overwrite_existing=True)

Movie created at KE_nw2_0.125deg_N15_KHGL800_hbbl5.mp4
